In [149]:
import pandas as pd
import os
import shutil
from PIL import Image
import cv2

In [150]:
iou = 0.4
depth = 30

In [152]:
df = pd.read_csv("/home/rinat/Downloads/videos/sample/sample.csv")

In [153]:
df

,video_path,frame_number,file_path,class,score,box
0,/home/rinat/Downloads/videos/sample.mp4,13,/home/rinat/Downloads/videos/sample/13toilet.jpg,toilet,0.304131,[ 203.14862 904.9939 552.88275 1145.0543 ]
1,/home/rinat/Downloads/videos/sample.mp4,193,/home/rinat/Downloads/videos/sample/193person.jpg,person,0.475293,[ -1.6867688 804.8532 45.797142 1000.9...
2,/home/rinat/Downloads/videos/sample.mp4,194,/home/rinat/Downloads/videos/sample/194person.jpg,person,0.510261,[-2.9765546e-01 8.0339648e+02 4.9709732e+01 ...
3,/home/rinat/Downloads/videos/sample.mp4,195,/home/rinat/Downloads/videos/sample/195person.jpg,person,0.361448,[-1.6463935e-01 8.0049139e+02 5.2283707e+01 ...
4,/home/rinat/Downloads/videos/sample.mp4,197,/home/rinat/Downloads/videos/sample/197suitcas...,suitcase,0.302059,[ 205.00635 892.9513 536.2966 1196.7872 ]
5,/home/rinat/Downloads/videos/sample.mp4,198,/home/rinat/Downloads/videos/sample/198suitcas...,suitcase,0.324279,[ 205.48592 893.13385 535.9672 1197.0281 ]
6,/home/rinat/Downloads/videos/sample.mp4,199,/home/rinat/Downloads/videos/sample/199suitcas...,suitcase,0.323054,[ 205.17252 893.2792 534.93756 1195.7728 ]
7,/home/rinat/Downloads/videos/sample.mp4,200,/home/rinat/Downloads/videos/sample/200suitcas...,suitcase,0.344457,[ 204.6335 892.49365 535.0691 1195.9733 ]
8,/home/rinat/Downloads/videos/sample.mp4,201,/home/rinat/Downloads/videos/sample/201suitcas...,suitcase,0.410775,[ 203.94327 892.8025 534.91815 1198.2689 ]
9,/home/rinat/Downloads/videos/sample.mp4,202,/home/rinat/Downloads/videos/sample/202suitcas...,suitcase,0.385586,[ 203.88136 892.95074 534.99896 1198.4059 ]


In [154]:
def add_cluster_members(index, df, cluster_number, depth, iou):
    row = df.loc[[index]]
    for i in range(depth):
        candidate_rows = df[(df['frame_number'] == row.iloc[0]['frame_number'] + i + 1) & 
                           (df['class'] == row.iloc[0]['class'])]
        if not candidate_rows.empty:
            row_iou = calculate_highest_iou(row, candidate_rows)
            if row_iou and row_iou['iou'] >= iou:
                df.loc[row_iou['row'].index[0], 'cluster'] = cluster_number
                add_cluster_members(row_iou['row'].index[0], df, cluster_number, depth, iou)
                break

def calculate_highest_iou(row, candidate_rows):
    row_iou = {}
    iou_candidate = 0
    for i in range(candidate_rows.shape[0]):
        candidate = candidate_rows.iloc[[i]]
        if iou_candidate < calculate_iou(row, candidate):
            iou_candidate = calculate_iou(row, candidate)
            row_iou['row'] = candidate
            row_iou['iou'] = iou_candidate
    return row_iou

def calculate_iou(row, candidate):
    boxA = string_to_list(row.iloc[0]['box'])
    boxB = string_to_list(candidate.iloc[0]['box'])

    #Following code was copied from https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def string_to_list(box):
    box = box[1:][:-1].split()
    return [float(i) for i in box]

In [155]:
df_clusters = df.copy()
df_clusters['cluster'] = -1

for i in range(df_clusters.shape[0]):
    if df_clusters.loc[i, 'cluster'] == -1:
        df_clusters.loc[i, 'cluster'] = i
        add_cluster_members(i, df_clusters, i, depth, iou)

In [156]:
df_clusters

,video_path,frame_number,file_path,class,score,box,cluster
0,/home/rinat/Downloads/videos/sample.mp4,13,/home/rinat/Downloads/videos/sample/13toilet.jpg,toilet,0.304131,[ 203.14862 904.9939 552.88275 1145.0543 ],0
1,/home/rinat/Downloads/videos/sample.mp4,193,/home/rinat/Downloads/videos/sample/193person.jpg,person,0.475293,[ -1.6867688 804.8532 45.797142 1000.9...,1
2,/home/rinat/Downloads/videos/sample.mp4,194,/home/rinat/Downloads/videos/sample/194person.jpg,person,0.510261,[-2.9765546e-01 8.0339648e+02 4.9709732e+01 ...,1
3,/home/rinat/Downloads/videos/sample.mp4,195,/home/rinat/Downloads/videos/sample/195person.jpg,person,0.361448,[-1.6463935e-01 8.0049139e+02 5.2283707e+01 ...,1
4,/home/rinat/Downloads/videos/sample.mp4,197,/home/rinat/Downloads/videos/sample/197suitcas...,suitcase,0.302059,[ 205.00635 892.9513 536.2966 1196.7872 ],4
5,/home/rinat/Downloads/videos/sample.mp4,198,/home/rinat/Downloads/videos/sample/198suitcas...,suitcase,0.324279,[ 205.48592 893.13385 535.9672 1197.0281 ],4
6,/home/rinat/Downloads/videos/sample.mp4,199,/home/rinat/Downloads/videos/sample/199suitcas...,suitcase,0.323054,[ 205.17252 893.2792 534.93756 1195.7728 ],4
7,/home/rinat/Downloads/videos/sample.mp4,200,/home/rinat/Downloads/videos/sample/200suitcas...,suitcase,0.344457,[ 204.6335 892.49365 535.0691 1195.9733 ],4
8,/home/rinat/Downloads/videos/sample.mp4,201,/home/rinat/Downloads/videos/sample/201suitcas...,suitcase,0.410775,[ 203.94327 892.8025 534.91815 1198.2689 ],4
9,/home/rinat/Downloads/videos/sample.mp4,202,/home/rinat/Downloads/videos/sample/202suitcas...,suitcase,0.385586,[ 203.88136 892.95074 534.99896 1198.4059 ],4


In [157]:
df_clusters.to_csv("/home/rinat/Downloads/videos/sample/sample_clusters.csv")

In [158]:
df_filtered = df_clusters[df_clusters['cluster'] != -1].copy()

In [159]:
df_filtered

,video_path,frame_number,file_path,class,score,box,cluster
0,/home/rinat/Downloads/videos/sample.mp4,13,/home/rinat/Downloads/videos/sample/13toilet.jpg,toilet,0.304131,[ 203.14862 904.9939 552.88275 1145.0543 ],0
1,/home/rinat/Downloads/videos/sample.mp4,193,/home/rinat/Downloads/videos/sample/193person.jpg,person,0.475293,[ -1.6867688 804.8532 45.797142 1000.9...,1
2,/home/rinat/Downloads/videos/sample.mp4,194,/home/rinat/Downloads/videos/sample/194person.jpg,person,0.510261,[-2.9765546e-01 8.0339648e+02 4.9709732e+01 ...,1
3,/home/rinat/Downloads/videos/sample.mp4,195,/home/rinat/Downloads/videos/sample/195person.jpg,person,0.361448,[-1.6463935e-01 8.0049139e+02 5.2283707e+01 ...,1
4,/home/rinat/Downloads/videos/sample.mp4,197,/home/rinat/Downloads/videos/sample/197suitcas...,suitcase,0.302059,[ 205.00635 892.9513 536.2966 1196.7872 ],4
5,/home/rinat/Downloads/videos/sample.mp4,198,/home/rinat/Downloads/videos/sample/198suitcas...,suitcase,0.324279,[ 205.48592 893.13385 535.9672 1197.0281 ],4
6,/home/rinat/Downloads/videos/sample.mp4,199,/home/rinat/Downloads/videos/sample/199suitcas...,suitcase,0.323054,[ 205.17252 893.2792 534.93756 1195.7728 ],4
7,/home/rinat/Downloads/videos/sample.mp4,200,/home/rinat/Downloads/videos/sample/200suitcas...,suitcase,0.344457,[ 204.6335 892.49365 535.0691 1195.9733 ],4
8,/home/rinat/Downloads/videos/sample.mp4,201,/home/rinat/Downloads/videos/sample/201suitcas...,suitcase,0.410775,[ 203.94327 892.8025 534.91815 1198.2689 ],4
9,/home/rinat/Downloads/videos/sample.mp4,202,/home/rinat/Downloads/videos/sample/202suitcas...,suitcase,0.385586,[ 203.88136 892.95074 534.99896 1198.4059 ],4


In [160]:
df_filtered = df_filtered[(df_clusters['class'] == 'person')]
                          #| 
                          #(df_clusters['class'] == 'car') | 
                          #(df_clusters['class'] == 'truck') |
                          #(df_clusters['class'] == 'bus')]

In [161]:
df_filtered_by_frequency = df_filtered.copy()

In [162]:
df_filtered_by_frequency.groupby('cluster').size()

cluster
1         3
11      297
76        2
100     278
825      11
838      20
855      52
919      55
1087    130
1129    140
1593      7
1618      4
1631      3
1640      7
1661      7
1684      8
1712     34
1717     43
1846     69
1885     67
2029     13
2049      9
2058     18
2073     37
2084    142
2152      5
2191    278
2351    601
2962      2
3021      2
3060    425
4025      1
4470      6
4489      4
4497    190
4625     11
4879     90
5069     14
5110     16
5160     57
5209     34
5275     34
5423     24
5469    204
5508     17
5580      5
5816    153
6381     12
6400      9
dtype: int64

In [126]:
df_filtered_by_frequency['cluster_size'] =  df_filtered_by_frequency.groupby('cluster')['cluster'].transform('size')

In [127]:
df_filtered_by_frequency = df_filtered_by_frequency[(df_filtered_by_frequency['cluster_size'] >= 6)]

In [128]:
df_normal = df_filtered_by_frequency.copy()

In [129]:
df_normal['file_path'] = df_normal['file_path'].str.replace(r'../wonderfull_life/wl_bw', '/home/rinat/wl/wl_bw', regex=True)

In [130]:
df_normal.groupby('cluster').size()

cluster
4488     65
6550      8
60975    17
dtype: int64

In [132]:
def save_df_files(df, folder, file_path = 'file_path', subfolder = None):
    for index, row in df.iterrows():
        file_name = row[file_path].split('/')[-1]
        directory = folder
        if subfolder:
            directory = directory + '/' + str(row[subfolder])
        if not os.path.exists(directory):
            os.makedirs(directory)
        shutil.copyfile(row[file_path], directory + '/' + file_name)

In [133]:
# Sort by cluster and frame number before using

def cluster_video(df, folder, file_path = 'file_path', video_col = 'cluster'):
    cluster = -1
    video = None
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    for index, row in df.iterrows():
        img = cv2.imread(row[file_path])
        if row[video_col] !=  cluster:
            if video is not None:
                video.release()
            height, width, layers = img.shape
            print('-----Enter cluster-------')
            print(img[0][0])
            if not os.path.exists(folder):
                os.makedirs(folder)
            video_path = folder + '/' + str(row[video_col]) + '.mp4'
            print(video_path)
            video = cv2.VideoWriter(video_path, fourcc, 10, (width, height))
            print(video)
            cluster = row[video_col]
        
        print('------------')
        print(img[0][0])
        video.write(img)

    video.release()

In [163]:
# Sort by cluster and frame number before using

def make_video(df, folder, file_path = 'file_path', video_col = 'analyzed'):
    img = cv2.imread(df.loc[0, file_path])
    height, width, layers = img.shape
    if not os.path.exists(folder):
        os.makedirs(folder)
    video_path = folder + '/' + str(row[video_col]) + '.mp4'
    print(video_path)
    video = cv2.VideoWriter(video_path, fourcc, 10, (width, height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    for index, row in df.iterrows():
        img = cv2.imread(row[file_path])
        video.write(img)
    video.release()

In [134]:
def crop_df_files(df, folder, crop, source_prefix = '', file_path = 'file_path', subfolder = None):
    for index, row in df.iterrows():
        file_name = row[file_path].split('/')[-1]
        directory = folder
        if subfolder:
            directory = directory + '/' + str(row[subfolder])
        if not os.path.exists(directory):
            os.makedirs(directory)
        im = Image.open(source_prefix + row[file_path])
        top, left, bottom, right = string_to_list(row[crop])
        crop_im = im.crop((left, top, right, bottom))
        crop_im.save( directory + '/' + file_name, "JPEG")

In [34]:
crop_df_files(df_filtered_by_frequency, '/home/rinat/yolo_demo/filtered_cars_croped', source_prefix = '/home/rinat/keras-yolo3/', crop = 'box', subfolder = 'cluster')

In [97]:
!ls

data  temporal_cluster.ipynb  temporal_cluster.py


In [135]:
save_df_files(df_normal, '../data/wl_analysis/filtered_benches_clustered', file_path = 'file_path', subfolder = 'cluster')

In [136]:
cluster_video(df_normal, '../data/wl_analysis/benches_video', file_path = 'file_path', video_col = 'cluster')

-----Enter cluster-------
[164 164 164]
../data/wl_analysis/benches_video/4488.mp4
<VideoWriter 0x7f476c6240b0>
------------
[164 164 164]
------------
[164 164 164]
------------
[164 164 164]
------------
[161 161 161]
------------
[162 162 162]
------------
[162 162 162]
------------
[160 160 160]
------------
[160 160 160]
------------
[160 160 160]
------------
[162 162 162]
------------
[162 162 162]
------------
[162 162 162]
------------
[158 158 158]
------------
[158 158 158]
------------
[158 158 158]
------------
[153 153 153]
------------
[155 155 155]
------------
[155 155 155]
------------
[155 155 155]
------------
[155 155 155]
------------
[154 154 154]
------------
[154 154 154]
------------
[153 153 153]
------------
[153 153 153]
------------
[155 155 155]
------------
[155 155 155]
------------
[165 165 165]
------------
[165 165 165]
------------
[165 165 165]
------------
[160 160 160]
------------
[152 152 152]
------------
[152 152 152]
------------
[152 152 15

In [103]:
image = Image.open('/home/rinat/wl/wl_bw/7368truck_train_bus_car_person_person_person.jpg')

In [101]:
print(image)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=950x720 at 0x7F476C5DD1D0>


In [104]:
im = cv2.imread('/home/rinat/wl/wl_bw/7368truck_train_bus_car_person_person_person.jpg')

In [105]:
im

array([[[165, 165, 165],
        [164, 164, 164],
        [154, 154, 154],
        ...,
        [100, 105, 104],
        [ 81,  86,  85],
        [ 66,  71,  70]],

       [[166, 166, 166],
        [166, 166, 166],
        [156, 156, 156],
        ...,
        [108, 113, 112],
        [ 87,  92,  91],
        [ 69,  74,  73]],

       [[162, 162, 162],
        [162, 162, 162],
        [152, 152, 152],
        ...,
        [101, 106, 105],
        [ 78,  83,  82],
        [ 60,  65,  64]],

       ...,

       [[142, 142, 142],
        [144, 144, 144],
        [146, 146, 146],
        ...,
        [ 62,  62,  62],
        [ 62,  62,  62],
        [ 62,  62,  62]],

       [[143, 143, 143],
        [145, 145, 145],
        [148, 148, 148],
        ...,
        [ 68,  68,  68],
        [ 68,  68,  68],
        [ 68,  68,  68]],

       [[145, 145, 145],
        [147, 147, 147],
        [150, 150, 150],
        ...,
        [ 73,  73,  73],
        [ 73,  73,  73],
        [ 73,  73,  73]]

In [164]:
cluster_video(df_normal, '/home/rinat/Downloads/videos/sample/', file_path = 'file_path')

-----Enter cluster-------
[164 164 164]
/home/rinat/Downloads/videos/sample//4488.mp4
<VideoWriter 0x7f4781cff0f0>
------------
[164 164 164]
------------
[164 164 164]
------------
[164 164 164]
------------
[161 161 161]
------------
[162 162 162]
------------
[162 162 162]
------------
[160 160 160]
------------
[160 160 160]
------------
[160 160 160]
------------
[162 162 162]
------------
[162 162 162]
------------
[162 162 162]
------------
[158 158 158]
------------
[158 158 158]
------------
[158 158 158]
------------
[153 153 153]
------------
[155 155 155]
------------
[155 155 155]
------------
[155 155 155]
------------
[155 155 155]
------------
[154 154 154]
------------
[154 154 154]
------------
[153 153 153]
------------
[153 153 153]
------------
[155 155 155]
------------
[155 155 155]
------------
[165 165 165]
------------
[165 165 165]
------------
[165 165 165]
------------
[160 160 160]
------------
[152 152 152]
------------
[152 152 152]
------------
[152 152